# ERP calculation using mne
### import


In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.io as scio
import os
import mne


## Load Task file

In [3]:
list_files_path_name = pd.read_csv('../matlab/task_track_file.csv',usecols=['local_file_path','preprocessed_one','ERP_npy_file','ERP_nobaseline_npy_file'])
num_task_file = list_files_path_name.shape[0]
list_files_path_name

,local_file_path,preprocessed_one,ERP_npy_file,ERP_nobaseline_npy_file
0,sub-001\ses-01\eeg\,pre_sub-001_ses-01_task-Rest_eeg.set,ERP_correct_sub-001_ses-01_task-Rest_eeg.npy,ERP_nb_correct_sub-001_ses-01_task-Rest_eeg.npy
1,sub-001\ses-02\eeg\,pre_sub-001_ses-02_task-Rest_eeg.set,ERP_correct_sub-001_ses-02_task-Rest_eeg.npy,ERP_nb_correct_sub-001_ses-02_task-Rest_eeg.npy
2,sub-002\ses-01\eeg\,pre_sub-002_ses-01_task-Rest_eeg.set,ERP_correct_sub-002_ses-01_task-Rest_eeg.npy,ERP_nb_correct_sub-002_ses-01_task-Rest_eeg.npy
3,sub-002\ses-02\eeg\,pre_sub-002_ses-02_task-Rest_eeg.set,ERP_correct_sub-002_ses-02_task-Rest_eeg.npy,ERP_nb_correct_sub-002_ses-02_task-Rest_eeg.npy
4,sub-003\ses-01\eeg\,pre_sub-003_ses-01_task-Rest_eeg.set,ERP_correct_sub-003_ses-01_task-Rest_eeg.npy,ERP_nb_correct_sub-003_ses-01_task-Rest_eeg.npy
...,...,...,...,...
70,sub-046\ses-01\eeg\,pre_sub-046_ses-01_task-Rest_eeg.set,ERP_correct_sub-046_ses-01_task-Rest_eeg.npy,ERP_nb_correct_sub-046_ses-01_task-Rest_eeg.npy
71,sub-047\ses-01\eeg\,pre_sub-047_ses-01_task-Rest_eeg.set,ERP_correct_sub-047_ses-01_task-Rest_eeg.npy,ERP_nb_correct_sub-047_ses-01_task-Rest_eeg.npy
72,sub-048\ses-01\eeg\,pre_sub-048_ses-01_task-Rest_eeg.set,ERP_correct_sub-048_ses-01_task-Rest_eeg.npy,ERP_nb_correct_sub-048_ses-01_task-Rest_eeg.npy
73,sub-049\ses-01\eeg\,pre_sub-049_ses-01_task-Rest_eeg.set,ERP_correct_sub-049_ses-01_task-Rest_eeg.npy,ERP_nb_correct_sub-049_ses-01_task-Rest_eeg.npy


## Load EEG signals and ERP calculation
### Baseline Correction applied


In [ ]:
clear_output(wait=True)

Path = 'D:\Mastersharif\MasterProject\data\parkinsons-oddball'
montage = mne.channels.read_custom_montage('Standard-10-20-Cap81.locs')
dict_temp={'S200':5,'S201':6,'S202':7}
ERP = {'S200':[],'S201':[],'S202':[]}
ERP_array = np.zeros([3,63,601],dtype=np.float64)
for i in np.arange(num_task_file):
    Path_to_load_file = os.path.join(Path,list_files_path_name['local_file_path'][i],list_files_path_name['preprocessed_one'][i])
    Path_to_save_file = os.path.join(Path,list_files_path_name['local_file_path'][i],list_files_path_name['ERP_npy_file'][i])
    if not os.path.isfile(Path_to_save_file):
        EEG = mne.io.read_raw_eeglab(Path_to_load_file,eog=(),preload=True)

        events, event_dict = mne.events_from_annotations(EEG, verbose=0)
        t  = events[events[:,2]==5]
        # t1 = mne.pick_events(events,include=[5])
        t2 = {'S200':5}
        pochs_1 = mne.Epochs(EEG,events=t,event_id=t2,
                        tmin=0.250, tmax=1.450, baseline=(0.250,0.450),preload=True, verbose=0)
        ERP['S200'] = pochs_1['S200'].average().to_data_frame()
        del ERP['S200']['time']
        ERP_array[0,:,:] = np.transpose(ERP['S200'].to_numpy())
        
        t  = events[events[:,2]==6]
        t2 = {'S201':6}
        pochs_1 = mne.Epochs(EEG,events=t,event_id=t2,
                        tmin=0.250, tmax=1.450, baseline=(0.250,0.450),preload=True, verbose=0) 
        ERP['S201'] = pochs_1['S201'].average().to_data_frame() 
        del ERP['S201']['time']
        ERP_array[1,:,:] = np.transpose(ERP['S201'].to_numpy())

        t  = events[events[:,2]==7]
        t2 = {'S202':7}               
        pochs_1 = mne.Epochs(EEG,events=t,event_id=t2,
                        tmin=-0.2, tmax=1, baseline=(-0.2,0),preload=True, verbose=0)  
        ERP['S202'] = pochs_1['S202'].average().to_data_frame() 
        del ERP['S202']['time']
        ERP_array[2,:,:] = np.transpose(ERP['S202'].to_numpy())


        np.save(Path_to_save_file,ERP_array)



                                                 
        
            
# events[:,2]

    

### No baseline Correction


In [ ]:
Path = 'D:\Mastersharif\MasterProject\data\parkinsons-oddball'
# montage = mne.channels.read_custom_montage('Standard-10-20-Cap81.locs')
dict_temp={'S200':5,'S201':6,'S202':7}
ERP = {'S200':[],'S201':[],'S202':[]}
ERP_array = np.zeros([3,63,601],dtype=np.float64)
for i in np.arange(num_task_file):
    Path_to_load_file = os.path.join(Path,list_files_path_name['local_file_path'][i],list_files_path_name['preprocessed_one'][i])
    Path_to_save_file = os.path.join(Path,list_files_path_name['local_file_path'][i],list_files_path_name['ERP_nobaseline_npy_file'][i])
    if not os.path.isfile(Path_to_save_file):
        EEG = mne.io.read_raw_eeglab(Path_to_load_file,eog=(),preload=True)

        events, event_dict = mne.events_from_annotations(EEG, verbose=0)
        t  = events[events[:,2]==5]
        # t1 = mne.pick_events(events,include=[5])
        t2 = {'S200':5}
        pochs_1 = mne.Epochs(EEG,events=t,event_id=t2,
                        tmin=0.250, tmax=1.450,baseline = None,preload=True, verbose=0)
        ERP['S200'] = pochs_1['S200'].average().to_data_frame()
        del ERP['S200']['time']
        ERP_array[0,:,:] = np.transpose(ERP['S200'].to_numpy())
        
        t  = events[events[:,2]==6]
        t2 = {'S201':6}
        pochs_1 = mne.Epochs(EEG,events=t,event_id=t2,
                        tmin=0.250, tmax=1.450,baseline = None,preload=True, verbose=0) 
        ERP['S201'] = pochs_1['S201'].average().to_data_frame() 
        del ERP['S201']['time']
        ERP_array[1,:,:] = np.transpose(ERP['S201'].to_numpy())

        t  = events[events[:,2]==7]
        t2 = {'S202':7}               
        pochs_1 = mne.Epochs(EEG,events=t,event_id=t2,
                        tmin=-0.2, tmax=1,baseline = None,preload=True, verbose=0)  
        ERP['S202'] = pochs_1['S202'].average().to_data_frame() 
        del ERP['S202']['time']
        ERP_array[2,:,:] = np.transpose(ERP['S202'].to_numpy())


        np.save(Path_to_save_file,ERP_array)



                                                 
        
            
# events[:,2]

    

## Load and plot ERP

### Check and compare the result with matlab calculation



In [ ]:
Channel_num = 62
subject_num = 34


Path_to_save_file = os.path.join(Path,list_files_path_name['local_file_path'][subject_num],list_files_path_name['ERP_npy_file'][subject_num])
test_ERP_array = np.load(Path_to_save_file)

test_dict = scio.loadmat('..\matlab\\test_ERP.mat')
ERP_matlab = test_dict['test_ERP']


plt.plot(test_ERP_array[0,Channel_num,:])




plt.plot(ERP_matlab[0,:])
plt.legend(["ERP py", "ERP mat"])
plt.show()

